In [6]:
import tvm
from tvm import relay


def get_demo_mod():
    x = relay.var("x", shape=(64, 64), dtype="float32")
    y = relay.var("y", shape=(64, 64), dtype="float32")
    z = relay.var("z", shape=(64, 64), dtype="float32")
    add = relay.add(x, y)
    subtract = relay.subtract(add, z)
    multiply = relay.multiply(subtract, z)

    func = relay.Function([x, y, z], multiply)
    mod = tvm.IRModule.from_expr(func)
    mod = relay.transform.InferType()(mod)
    return mod


mod = get_demo_mod()
print(mod)

def @main(%x: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */, %y: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */, %z: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */) -> Tensor[(64, 64), float32] {
  %0 = add(%x, %y) /* ty=Tensor[(64, 64), float32] */;
  %1 = subtract(%0, %z) /* ty=Tensor[(64, 64), float32] */;
  multiply(%1, %z) /* ty=Tensor[(64, 64), float32] */
}



[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:382: InferType: Executing module pass with opt level: 0


In [7]:
with tvm.transform.PassContext(opt_level=2):
    graph, lib, params = relay.build(mod, target="c", params=None)

print(graph)

[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: Running pass RemoveUnusedFunctions
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:382: Build: RemoveUnusedFunctions: Executing module pass with opt level: 1
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: Running pass ToBasicBlockNormalForm
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:382: Build: ToBasicBlockNormalForm: Executing module pass with opt level: 1
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: Running pass qnn.Legalize
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: Running pass QnnLegalize
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:382: Build: InferType: Executing module pass with opt level: 0
[19:05:02] /home/qzylalala/work_space/tvm/src/relay/ir/transform.cc:124: Build: QnnLegalize: Executing function pass with opt level: 1
[19:05:02] /home/qzylalala/work_space/tvm/src/r

{
  "nodes": [
    {
      "op": "null", 
      "name": "x", 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "y", 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "z", 
      "inputs": []
    }, 
    {
      "op": "tvm_op", 
      "name": "tvmgen_default_fused_add_subtract_multiply", 
      "attrs": {
        "num_outputs": "1", 
        "num_inputs": "3", 
        "flatten_data": "0", 
        "func_name": "tvmgen_default_fused_add_subtract_multiply", 
        "hash": "8b8bdf1c4ac262e5"
      }, 
      "inputs": [
        [
          0, 
          0, 
          0
        ], 
        [
          1, 
          0, 
          0
        ], 
        [
          2, 
          0, 
          0
        ]
      ]
    }
  ], 
  "arg_nodes": [0, 1, 2], 
  "heads": [
    [
      3, 
      0, 
      0
    ]
  ], 
  "attrs": {
    "dltype": [
      "list_str", 
      [
        "float32", 
        "float32", 
        "float32", 
        "float32"
      ]
    ],

[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: GraphExecutorCodegen: LowerTE: LowerTensorExpr: Running pass tir.RemoveNoOp
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: GraphExecutorCodegen: LowerTE: LowerTensorExpr: Running pass tir.RewriteUnsafeSelect
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: GraphExecutorCodegen: LowerTE: LowerTensorExpr: Running pass tir.HoistIfThenElse
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: GraphExecutorCodegen: LowerTE: LowerTensorExpr: Running pass tir.InsertHoistIfThenElse
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: GraphExecutorCodegen: LowerTE: LowerTensorExpr: Running pass tir.Simplify
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: GraphExecutorCodegen: LowerTE: LowerTensorExpr: Running pass tir.RemoveNoOp
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: Gra

In [8]:
print(lib.get_source())

// tvm target: c -keys=cpu 
#define TVM_EXPORTS
#include "tvm/runtime/c_runtime_api.h"
#include "tvm/runtime/c_backend_api.h"
#include <math.h>
#include <stdbool.h>
#ifdef __cplusplus
extern "C"
#endif
TVM_DLL int32_t tvmgen_default_fused_add_subtract_multiply(void* args, int32_t* arg_type_ids, int32_t num_args, void* out_ret_value, int32_t* out_ret_tcode, void* resource_handle) {
  void* arg_p0 = (((TVMValue*)args)[0].v_handle);
  int32_t arg_p0_code = arg_type_ids[0];
  void* arg_p1 = (((TVMValue*)args)[1].v_handle);
  int32_t arg_p1_code = arg_type_ids[1];
  void* arg_p2 = (((TVMValue*)args)[2].v_handle);
  int32_t arg_p2_code = arg_type_ids[2];
  void* arg_T_multiply = (((TVMValue*)args)[3].v_handle);
  int32_t arg_T_multiply_code = arg_type_ids[3];
  void* p0 = (((DLTensor*)arg_p0)[0].data);
  void* arg_p0_shape = (((DLTensor*)arg_p0)[0].shape);
  void* arg_p0_strides = (((DLTensor*)arg_p0)[0].strides);
  int32_t dev_id = (((DLTensor*)arg_p0)[0].device.device_id);
  void* p1 = (((

In [9]:
from tvm.relay import transform

if not tvm.get_global_func("relay.ext.PIMCompiler", True):
    print("skip because PIM codegen is not available")
else:
    print("PIM codegen is now available")

mod = get_demo_mod()
mod = relay.transform.AnnotateTarget("pim")(mod)
mod = transform.MergeCompilerRegions()(mod)
mod = relay.transform.PartitionGraph()(mod)
print(mod)

skip because PIM codegen is not available
def @main(%x: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */, %y: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */, %z: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */) -> Tensor[(64, 64), float32] {
  @tvmgen_default_pim_main_0(%x, %y, %z) /* ty=Tensor[(64, 64), float32] */
}

def @tvmgen_default_pim_main_0(%pim_0_i0: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */, %pim_0_i1: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */, %pim_0_i2: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */, Compiler="pim", Primitive=1, Inline=1, global_symbol="tvmgen_default_pim_main_0") -> Tensor[(64, 64), float32] {
  %0 = add(%pim_0_i0, %pim_0_i1) /* ty=Tensor[(64, 64), float32] */;
  %1 = subtract(%0, %pim_0_i2) /* ty=Tensor[(64, 64), float32] */;
  multiply(%1, %pim_0_i2) /* ty=Tensor[(64, 64), float32] */
}



[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:382: InferType: Executing module pass with opt level: 0
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Running pass AnnotateTargetFunc
[19:05:02] /home/qzylalala/work_space/tvm/src/ir/transform.cc:382: InferType: Executing module pass with opt level: 0
[19:05:02] /home/qzylalala/work_space/tvm/src/relay/ir/transform.cc:124: AnnotateTargetFunc: Executing function pass with opt level: 0
[19:05:02] /home/qzylalala/work_space/tvm/src/relay/ir/transform.cc:125: AnnotateTargetFunc: Input module:
def @main(%x: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */, %y: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */, %z: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */) -> Tensor[(64, 64), float32] {
  %0 = add(%x, %y) /* ty=Tensor[(64, 64), float32] */;
  %1 = subtract(%0, %z) /* ty=Tensor[(64, 64), float32] */;
  multiply(%1, %z) /* ty=Tensor[(64, 64), float32] */
}

[19:05:02] /

In [10]:
with tvm.transform.PassContext(opt_level=2):
    graph, lib, params = relay.build(mod, target="c", params=None)

# lib.export_library("libpim.so")
print(lib.imported_modules[0].get_source())
print(lib.imported_modules[1].get_source())

[19:05:03] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: Running pass RemoveUnusedFunctions
[19:05:03] /home/qzylalala/work_space/tvm/src/ir/transform.cc:382: Build: RemoveUnusedFunctions: Executing module pass with opt level: 1
[19:05:03] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: Running pass ToBasicBlockNormalForm
[19:05:03] /home/qzylalala/work_space/tvm/src/ir/transform.cc:382: Build: ToBasicBlockNormalForm: Executing module pass with opt level: 1
[19:05:03] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: Running pass qnn.Legalize
[19:05:03] /home/qzylalala/work_space/tvm/src/ir/transform.cc:443: Build: Running pass QnnLegalize
[19:05:03] /home/qzylalala/work_space/tvm/src/ir/transform.cc:382: Build: InferType: Executing module pass with opt level: 0
[19:05:03] /home/qzylalala/work_space/tvm/src/relay/ir/transform.cc:124: Build: QnnLegalize: Executing function pass with opt level: 1
[19:05:03] /home/qzylalala/work_space/tvm/src/r

;
#include <cstdint>
#include <iostream>
#include <cstdlib>
#include <stdio.h>
#include <cstring>
#include <tvm/runtime/c_runtime_api.h>
#include <tvm/runtime/packed_func.h>
#include <dlpack/dlpack.h>
#include <dnnl/dnnl_kernel.h>
using namespace tvm::runtime;
using namespace tvm::runtime::contrib;


  #define CSOURCE_BINARY_OP_1D(p_ID_, p_OP_, p_DIM1_)       \
    extern "C" void p_ID_(float* a, float* b, float* out) { \
      for (int64_t i = 0; i < p_DIM1_; ++i) {               \
        out[i] = a[i] p_OP_ b[i];                           \
      }                                                     \
    }

  #define CSOURCE_BINARY_OP_2D(p_ID_, p_OP_, p_DIM1_, p_DIM2_)  \
    extern "C" void p_ID_(float* a, float* b, float* out) {     \
      for (int64_t i = 0; i < p_DIM1_; ++i) {                   \
        for (int64_t j = 0; j < p_DIM2_; ++j) {                 \
          int64_t k = i * p_DIM2_ + j;                          \
          out[k] = a[k] p_OP_ b[k];                

[19:05:03] /home/qzylalala/work_space/tvm/src/relay/ir/transform.cc:125: Build: GraphExecutorCodegen: LowerTE: LowerTensorExpr: Input module:
def @main(%x {virtual_device=VirtualDevice(device_type=1, virtual_device_id=0, target=Target(id=3e3ec00, kind='c', keys={'cpu'}, host=Target(id=35be740, kind='c', keys={'cpu'})))}: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */, %y {virtual_device=VirtualDevice(device_type=1, virtual_device_id=0, target=Target(id=3e3ec00, kind='c', keys={'cpu'}, host=Target(id=35be740, kind='c', keys={'cpu'})))}: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */, %z {virtual_device=VirtualDevice(device_type=1, virtual_device_id=0, target=Target(id=3e3ec00, kind='c', keys={'cpu'}, host=Target(id=35be740, kind='c', keys={'cpu'})))}: Tensor[(64, 64), float32] /* ty=Tensor[(64, 64), float32] */, executor=meta[Executor][0], runtime=meta[Runtime][0], hash="7202c58eea5f7051", virtual_device=VirtualDevice(device_type=1, virtual_device_id=0, targe